In [2]:
import os
import pickle
import tarfile
import datetime
import numpy as np
import urllib.request
import sklearn.metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import visualkeras

ModuleNotFoundError: No module named 'visualkeras'

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
%load_ext tensorboard

In [5]:
HISTORY_DIR = './history'
os.makedirs(HISTORY_DIR, exist_ok=True)

In [6]:
def get_data():
    if not os.path.exists('cifar-10-batches-py/'):
        urllib.request.urlretrieve('https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', 'cifar-10-python.tar.gz')
        file = tarfile.open('cifar-10-python.tar.gz', 'r:gz')
        file.extractall()
        
    X, y = [], []
    for i in range(1, 6):
        d = pickle.load(open(os.path.join('cifar-10-batches-py', f'data_batch_{i}'), 'rb'), encoding='bytes')
        X.append(d[b'data'])
        y.append(d[b'labels'])
        
    d = pickle.load(open(os.path.join('cifar-10-batches-py', 'test_batch'), 'rb'), encoding='bytes')
        
    return (
        np.concatenate(X, axis=0).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1),
        np.concatenate(y, axis=0),
        pickle.load(open('cifar-10-batches-py/batches.meta', 'rb'))['label_names'],
        np.array(d[b'data']).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1),
        np.array(d[b'labels'])
    )

In [7]:
def plot_confusion_matrix(model: tf.keras.models.Model, X: np.ndarray, y: np.ndarray, labels: list[str], batch_size: int = 8, **kwargs):
    sklearn.metrics.ConfusionMatrixDisplay.from_predictions(
        y,
        model.predict(X, verbose=False, batch_size=batch_size)[output_index].argmax(axis=-1),
        display_labels=labels,
        xticks_rotation='vertical',
        **kwargs
    )

In [8]:
def plot_loss(model):
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation', 'Accuracy', 'Val accuracy'])
    plt.show()

In [9]:
X, y, labels, X_test, y_test = get_data()

X_train = tf.keras.utils.normalize(X, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

In [10]:
X_train.shape

(50000, 32, 32, 3)

In [11]:
y.shape

(50000,)

In [13]:
from keras import layers

model = tf.keras.models.Sequential([
    layers.InputLayer(input_shape=(32, 32, 3)),
    layers.Conv3D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv3D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # -> 16

    layers.Conv2D(128, (4, 4), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Conv2D(128, (4, 4), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # -> 8

    layers.Conv2D(256, (4, 4), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Conv2D(256, (4, 4), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    # -> 4

    # layers.Dropout(0.25),
    # layers.Conv2D(128, (3, 3), activation='relu'),
    # layers.BatchNormalization(),
    # layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(10, activation='softmax')
])

visualkeras.layered_view(model, legend=True)

ValueError: Input 0 of layer "conv3d_1" is incompatible with the layer: expected min_ndim=5, found ndim=4. Full shape received: (None, 32, 32, 3)

In [79]:
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_137 (Conv2D)         (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_175 (Ba  (None, 32, 32, 64)       256       
 tchNormalization)                                               
                                                                 
 dropout_109 (Dropout)       (None, 32, 32, 64)        0         
                                                                 
 conv2d_138 (Conv2D)         (None, 32, 32, 128)       73856     
                                                                 
 batch_normalization_176 (Ba  (None, 32, 32, 128)      512       
 tchNormalization)                                               
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 16, 16, 128)    

In [80]:
y.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [81]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [82]:
logdir = os.path.join(HISTORY_DIR, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [83]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(logdir, 'model'),
    save_best_only=True
)

In [84]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(logdir, 'logs'),    
)

In [85]:
%tensorboard --logdir $logdir

Launching TensorBoard...

In [86]:
model.fit(X_train, y, validation_data=(X_test, y_test), batch_size=32, epochs=5, callbacks=[model_checkpoint_callback, tensorboard_callback])

Epoch 1/5
  10/1563 [..............................] - ETA: 26:03 - loss: 3.2978 - accuracy: 0.1562

KeyboardInterrupt: 

In [52]:
plot_loss(model.history)

KeyError: 'loss'

In [ ]:
plot_confusion_matrix(model, X, y, labels=labels)

In [ ]:
plot_confusion_matrix(model, X_test, y_test, labels=labels)